In [ ]:
import ssl
from pyVim import connect
from pyVmomi import vmodl
from pyVmomi import vim

# 虚拟机扫描

sslContext = ssl.SSLContext(ssl.PROTOCOL_SSLv23)
sslContext.verify_mode = ssl.CERT_NONE

service_instance = connect.SmartConnect(host='10.40.2.77',
                                        user='root',
                                        pwd='0i!od4r[GN52!bxoa0',
                                        port=443,
                                        sslContext=sslContext)
content = service_instance.RetrieveContent()

# ---------------------------------------------------------------------
containerView = content.viewManager.CreateContainerView(
    content.rootFolder, [vim.VirtualMachine], True)

children = containerView.view
for child in children:
    print child, type(child)
else:
    summary = child.summary
#     print summary.config
#     print summary.runtime
    print summary.guest, dir(summary.guest)
    print summary.guest.ipAddress
#     print dir(child)
   
# ---------------------------------------------------------------------
objview = content.viewManager.CreateContainerView(content.rootFolder,
                                                  [vim.HostSystem],
                                                  True)
esxi_hosts = objview.view
objview.Destroy()

datastores = {}
# 获取vmware 宿主机的相关硬件信息
for esxi_host in esxi_hosts:
    print 'esxi_host: ', esxi_host, type(esxi_host)
    print esxi_host.name
else:
    print dir(esxi_host)

# ---------------------------------------------------------------------
    
connect.Disconnect(service_instance)

In [1]:
from libnmap.process import NmapProcess
from libnmap.parser import NmapParser, NmapParserException

# 扫描网络端口

hosts_hash = []
option = '-sU -p 161,1106'
nm = NmapProcess(targets='10.30.0.0/24', options=option)
nm.run()
nmap_report = NmapParser.parse(nm.stdout)
for host in nmap_report.hosts:
    if host.is_up():
        for service in host.services:
            print service.state
            if service.state.startswith('open'):
                active_ip = ':'.join([host.address, str(service.port)])
                hosts_hash.append(active_ip)
        else:
            print dir(service)
    else:
#         print '--', host
        pass

print 'hosts_hash', hosts_hash
    

closed
open|filtered
['__class__', '__delattr__', '__dict__', '__doc__', '__eq__', '__format__', '__getattribute__', '__hash__', '__init__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_cpelist', '_owner', '_portid', '_protocol', '_reason', '_reason_ip', '_reason_ttl', '_service', '_service_extras', '_servicefp', '_state', '_tunnel', 'banner', 'changed', 'cpelist', 'diff', 'get_dict', 'id', 'open', 'owner', 'port', 'protocol', 'reason', 'reason_ip', 'reason_ttl', 'scripts_results', 'service', 'service_dict', 'servicefp', 'state', 'tunnel']
closed
open|filtered
['__class__', '__delattr__', '__dict__', '__doc__', '__eq__', '__format__', '__getattribute__', '__hash__', '__init__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_cpelist', '_owner', '_portid', '_protocol', '_re

In [35]:
#!/usr/bin/env python
# coding: utf8

"""
project: CMDB_proxy
create date: 2018/5/5 
__author__ = xiashuangxi
"""


import re
import yaml
import netsnmp
from collections import defaultdict


class SessionClient(object):
    """扫描活跃IP的设备类型和需要的信息，返回信息字典"""

    def __init__(self, host, conf=CONF, name='session'):
        self.Host = host
        self.Conf = conf
        self.Name = name
        self.AuthProto = conf['snmpV3']['AuthProto']
        self.AuthPass = conf['snmpV3']['AuthPass']
        self.PrivPass = conf['snmpV3']['PrivPass']
        self.PrivProto = conf['snmpV3']['PrivProto']
        self.SecLevel = conf['snmpV3']['SecLevel']
        self.SecName = conf['snmpV3']['SecName']
        self.Timeout = conf['snmpV3']['Timeout']
        self.PublicOid = conf['snmpV3']['oid']
        self.Mac_oid = conf['port_to_mac']
        self.Session = self.get_if_name_first()
        self.Var_list_0 = netsnmp.VarList(netsnmp.Varbind(self.Mac_oid[0]))  # 获取交换机端口号信息
        self.Var_list_1 = netsnmp.VarList(netsnmp.Varbind(self.Mac_oid[1]))  # 获取交换机端口号对应的接口索引
        self.Var_list_2 = netsnmp.VarList(netsnmp.Varbind(self.Mac_oid[2]))  # 接口索引对应的二层接口索引
        self.Var_list_3 = netsnmp.VarList(netsnmp.Varbind(self.Mac_oid[3]))  # 二层接口索引对应的MAC地址

        self.New_Var_list_0 = self.Session.walk(self.Var_list_0)
        self.New_Var_list_1 = self.Session.walk(self.Var_list_1)
        self.New_Var_list_2 = self.Session.walk(self.Var_list_2)
        self.New_Var_list_3 = self.Session.walk(self.Var_list_3)

        self.res = {i.val: i.iid for i in self.Var_list_0}  # 生成{key=端口名称，value=端口号}的端口信息字典
        self.res1 = {i.val: i.tag[-1:] for i in self.Var_list_1}  # 生成{key=端口号，value=端口索引}的对应字典
        self.res2 = {i.val: '.'.join(i.tag.split('.')[-6:]) for i in self.Var_list_2}  # 生成{key=端口索引，value=二层接口索引}的对应字典
        self.res3 = {'.'.join(i.tag.split('.')[-6:]): ':'.join('%02x' % ord(k) for k in i.val) for i in self.Var_list_3}  # 生成{key=二层接口索引,
        # value=将原十进制的字符转换为mac地址 }的对应表

    def get_if_name_first(self):
        """实例化snmp客户端"""
        session = netsnmp.Session(Version=3,
                                  DestHost=self.Host,
                                  SecName=self.SecName,
                                  SecLevel=self.SecLevel,
                                  AuthProto=self.AuthProto,
                                  AuthPass=self.AuthPass,
                                  PrivProto=self.PrivProto,
                                  PrivPass=self.PrivPass,
                                  Timeout=int(self.Timeout))

        return session

    def check_network_asset_type(self):
        netasset_type = ''
        try:
            public_var_list = netsnmp.VarList(netsnmp.Varbind(self.PublicOid))
            asset_type_temp = self.Session.get(public_var_list)
            if asset_type_temp[0]:
                pattern = re.compile(r'[A-Z]+[0-9]{4,6}[A-Z]?')
                if re.search(pattern, asset_type_temp[0]):
                    search_value = re.search(pattern, asset_type_temp[0])
                    net_model = search_value.group()
                    netasset_type = self.get_network_type(net_model)
            return netasset_type
        except Exception as e:
            print '+++++++++++', e

        return netasset_type

    def get_network_type(self, value):
        type_string = value.upper()
        if re.search(r'^USG|FW', type_string):
            return 'firewall'
        elif re.search(r'^AR', type_string):
            return 'route'
        elif re.search(r'^CE', type_string):
            return 'ce_switch'
        elif re.search(r'^S', type_string):
            return 'switch'

    def result_back(self):
        result = defaultdict(dict)
        result['management_ip'] = self.Host
        asset_type = self.check_network_asset_type()
        result['asset_type'] = asset_type
        if asset_type:
            for k, v in self.Conf[asset_type].iteritems():
                print k, v
                Var_list = netsnmp.VarList(netsnmp.Varbind(v))
                oid_value = self.Session.get(Var_list)
                if k == 'model':
                    _temp = re.split('\r\n', oid_value[0])
                    result[k] = _temp[0].strip(' ')
                elif k in ("asset_type", "sn", "USG6650_sn"):
                    result[k] = oid_value[0]
                else:
                    result[k] = oid_value
                
                print oid_value
                    
#                 if k == 'ports':
#                     print '*' * 80
#                     print oid_value
#                     print '*' * 80
                
            result['mac'] = self.build_mac_dict()
        else:
            result['mac'] = ''
        
        # 防火墙中去掉 USG6650 类型资产
        if "USG6650_sn" in result:
            result['sn'] = result['USG6650_sn']
            del result['USG6650_sn']
        
        print '%s》》》:%s_%s_%s \r\n' % (self.Name, self.Host, asset_type, result)
        return self.Host, asset_type, result

    def build_mac_dict(self):
        dict_temp = defaultdict(dict)
        for k, v in self.res.items():
            if v in self.res1:
                _res1 = self.res1[v]
                if _res1 in self.res2:
                    _res2 = self.res2[_res1]
                    if _res2 in self.res3:
                        dict_temp[k] = self.res3[_res2]
        return dict_temp
    
    def get_network_port_count(self, value_list):
        def check_if_real_port(value):
            key_words = ['GigabitEthernet', 'GigabitEthernet', 'Ethernet3']
            for word in key_words:
                if value.index(word) != -1:
                    return True
            
            return False
        
        counts = 0
        for value in value_list:
            if check_if_real_port(value):
                counts += 1
        
        return counts


if __name__ == "__main__":
    CONF = yaml.load(file("conf/conf.yaml"))
    print CONF
    
    hosts = ['10.30.0.1:1106', '10.30.0.2:1106', '10.30.0.3:1106']
#     hosts = ['10.30.0.1:1106']
    for host in hosts: 
        print host
        session = SessionClient(host, CONF, '123')
        session.result_back()



{'network_segment': ['10.30.0.0/24'], 'log': {'logLevel': 'debug', 'logPath': './'}, 'ce_switch': {'asset_num': '.1.3.6.1.2.1.1.5.0', 'model': '.1.3.6.1.2.1.47.1.1.1.1.7.16842753', 'sn': '.1.3.6.1.2.1.47.1.1.1.1.11.16842753', 'ports': '.1.3.6.1.2.1.31.1.1.1.1'}, 'firewall': {'asset_num': '.1.3.6.1.2.1.1.5.0', 'model': '.1.3.6.1.2.1.47.1.1.1.1.7.67108867', 'ports': '.1.3.6.1.2.1.31.1.1.1.1', 'USG6650_sn': '.1.3.6.1.2.1.47.1.1.1.1.11.69468169', 'sn': '.1.3.6.1.2.1.47.1.1.1.1.11.67108873'}, 'route': {'asset_num': '.1.3.6.1.2.1.1.5.0', 'model': '.1.3.6.1.2.1.47.1.1.1.1.7.67108867', 'sn': '.1.3.6.1.2.1.47.1.1.1.1.11.67108867', 'ports': '.1.3.6.1.2.1.31.1.1.1.1'}, 'port_to_mac': ['.1.3.6.1.2.1.31.1.1.1.1', '.1.3.6.1.2.1.17.1.4.1.2', '.1.3.6.1.2.1.17.4.3.1.2', '.1.3.6.1.2.1.17.4.3.1.1'], 'switch': {'asset_num': '.1.3.6.1.2.1.1.5.0', 'model': '.1.3.6.1.2.1.1.1.0', 'sn': '.1.3.6.1.2.1.47.1.1.1.1.11.67108867', 'ports': '.1.3.6.1.2.1.31.1.1.1.1'}, 'gevent': {'nums': 1}, 'snmpV3': {'PrivPass': 'QU